In [1]:
# Importing dependencies
import os
from langchain_openai import ChatOpenAI
from crewai_tools import PDFSearchTool
from crewai_tools import tool


In [2]:
from crewai import Agent, Crew, Process, Task
from langchain.agents import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_huggingface import *

In [3]:
# set GROQ_API_KEY and TAVILY_API_KEY
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
#Define LLM 
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,

)


llm1=ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama-3.2-1b-preview",
    temperature=0.1,
    max_tokens=1000,)

In [5]:

# use attentions paper as sample
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response =requests.get(pdf_url)
with open('attentions_is_all_you_need.pdf','wb') as file:
  file.write(response.content)

In [6]:
## Create a RAG tool variable to pass our PDF


rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

/opt/anaconda3/envs/ths/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
result = rag_tool.run("How did self-attention mechanism evolve in large language models?")
print(result)

Using Tool: Search a PDF's content
Relevant Content:
This makes it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 22, 23, 19]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence- aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [28]. To

In [8]:
## Using Tavily API Key for search , try with DuckDuck Go
web_search_tool = TavilySearchResults(k=3)

In [9]:
web_search_tool.run("How did self-attention mechanism evolve in large language models?")

"HTTPError('429 Client Error: Too Many Requests for url: https://api.tavily.com/search')"

In [10]:
## Define a Tool
@tool
def router_tool(question):
  """Router"""
  if 'self-attention' in question:
    return 'vectorstore'

  else:
    return 'web-search'

In [11]:
#Agent 1

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 1
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [ ]:
#Agent 2
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)




#Task 2
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [ ]:
#Agent 3

Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 3
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [15]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent],
    tasks=[router_task,retriever_task],
    verbose=True

)

2024-11-14 21:40:15,326 - 8148842560 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [19]:
inputs ={"question":"How does self-attention mechanism help large language models?"}
#inputs ={"question":"How self-attention vs Computer vision"}


In [20]:
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How does self-attention mechanism help large language models?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question seems to be related to the concept of self-attention mechanism in large language models, which is a topic in the field of Retrieval-Augmented Generation.

Action: router_tool

Action Input: {"question": "How does self-attention mechanism help large language models?"}
 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task ex

In [21]:
result

'Self-attention mechanism helps large language models in several ways:\n\n* It allows the model to focus on specific parts of the input sequence that are relevant to the task at hand, rather than treating the entire sequence as a single entity.\n* It enables the model to capture long-range dependencies and relationships between different parts of the input sequence.\n* It helps the model to learn more nuanced and context-dependent representations of the input sequence.\n* It allows the model to handle input sequences of varying lengths and complexities.\n* It enables the model to learn more accurate and robust representations of the input sequence.\n\nThese benefits enable large language models to perform better on a wide range of natural language processing tasks, such as language translation, text summarization, and question answering.'